In [2]:
data = [  ["What can you see?", "vision"],
  ["What objects are in the image?", "vision"],
  ["Can you describe the scene?", "vision"],
  ["Can you detect a person in the image?", "vision"],
  ["Are there any cars present in the scene?", "vision"],
  ["Can you recognize a truck in the image?", "vision"],
  ["Is there any motorcycle in the picture?", "vision"],
  ["Are there any bicycles in the scene?", "vision"],
  ["Can you identify a bus in the image?", "vision"],
  ["Is there any train in the picture?", "vision"],
  ["Can you detect a traffic light in the scene?", "vision"],
  ["Are there any stop signs present in the image?", "vision"],
  ["Can you recognize a fire hydrant in the picture?", "vision"],
  ["Is there any parking meter in the scene?", "vision"],
  ["Can you detect a bench in the image?", "vision"],
  ["Are there any birds present in the picture?", "vision"],
  ["Can you recognize a cat in the scene?", "vision"],
  ["Is there any dog in the image?", "vision"],
  ["Can you detect a horse in the picture?", "vision"],
  ["Are there any sheep present in the scene?", "vision"],
  ["Can you recognize a cow in the image?", "vision"],
  ["Is there any elephant in the picture?", "vision"],
  ["Can you detect a bear in the scene?", "vision"],
  ["Are there any zebra present in the image?", "vision"],
  ["Can you recognize a giraffe in the picture?", "vision"],
  ["Is there any backpack in the scene?", "vision"],
  ["Can you detect an umbrella in the image?", "vision"],
  ["Are there any handbags present in the picture?", "vision"],
  ["Can you recognize a tie in the scene?", "vision"],
  ["Is there any suitcase in the image?", "vision"],
  ["Can you detect a frisbee in the picture?", "vision"],
  ["Are there any skis present in the scene?", "vision"],
  ["Can you recognize a snowboard in the image?", "vision"],
  ["Is there any sports ball in the picture?", "vision"],
  ["Can you detect a kite in the scene?", "vision"],
  ["Are there any baseball bats present in the image?", "vision"],
  ["Can you recognize a baseball glove in the picture?", "vision"],
  ["Is there any skateboard in the scene?", "vision"],
  ["Can you detect a surfboard in the image?", "vision"],
  ["Are there any tennis rackets present in the picture?", "vision"],
  ["Can you recognize a bottle in the scene?", "vision"],
  ["Is there any wine glass in the image?", "vision"],
  ["Can you detect a cup in the picture?", "vision"],
  ["Are there any forks present in the scene?", "vision"],
  ["Can you recognize a knife in the image?", "vision"],
  ["Is there any spoon in the picture?", "vision"],
  ["Can you detect a bowl in the scene?", "vision"],
  ["Are there any bananas present in the image?", "vision"],
  ["Can you recognize an apple in the picture?", "vision"],
  ["Is there any sandwich in the scene?", "vision"],
  ["How was your day today?", "chat"],
  ["What's your favorite hobby?", "chat"],
  ["Have you seen any good movies lately?", "chat"],
  ["What type of music do you enjoy listening to?", "chat"],
  ["Do you have any favorite books?", "chat"],
  ["What are your weekend plans?", "chat"],
  ["Have you ever traveled abroad?", "chat"],
  ["What's your favorite type of cuisine?", "chat"],
  ["Do you play any musical instruments?", "chat"],
  ["What's your favorite season?", "chat"],
  ["How do you usually spend your free time?", "chat"],
  ["Do you have any pets?", "chat"],
  ["What's your favorite subject in school?", "chat"],
  ["Do you enjoy sports?", "chat"],
  ["What's your favorite way to stay active?", "chat"],
  ["Are you a morning person or a night owl?", "chat"],
  ["What's your favorite holiday?", "chat"],
  ["Do you have any siblings?", "chat"],
  ["What's your dream job?", "chat"],
  ["If you could travel anywhere, where would you go?", "chat"],
  ["Do you prefer the city or the countryside?", "chat"],
  ["What's your favorite type of weather?", "chat"],
  ["Have you ever been camping?", "chat"],
  ["What's the most interesting thing you've learned recently?", "chat"],
  ["Do you enjoy trying new things?", "chat"],
  ["Are you an introvert or an extrovert?", "chat"],
  ["What's your favorite way to relax?", "chat"],
  ["Do you have a favorite TV show?", "chat"],
  ["What's your favorite childhood memory?", "chat"],
  ["Do you enjoy cooking?", "chat"],
  ["Are you into gardening?", "chat"],
  ["Do you have any favorite podcasts?", "chat"],
  ["What's your favorite type of art?", "chat"],
  ["Do you have a favorite color?", "chat"],
  ["What's something you've always wanted to learn?", "chat"],
  ["Have you ever been to a concert?", "chat"],
  ["Do you have any favorite quotes?", "chat"],
  ["What's your favorite type of dessert?", "chat"],
  ["Do you enjoy going to parties?", "chat"],
  ["What's your favorite kind of coffee?", "chat"],
  ["Do you prefer tea or coffee?", "chat"],
  ["What's your favorite type of exercise?", "chat"],
  ["Do you have a favorite board game?", "chat"],
  ["What's your favorite type of flower?", "chat"],
  ["Do you enjoy hiking?", "chat"],
  ["What's your favorite animal?", "chat"],
  ["Do you like going to the beach?", "chat"],
  ["What's your favorite way to stay organized?", "chat"],
  ["Do you have any hidden talents?", "chat"],
  ["What's something you're proud of?", "chat"],
  ["I went for a walk in the park today.", "chat"],
  ["I've been learning how to play the guitar.", "chat"],
  ["Yesterday, I tried a new recipe and it turned out great.", "chat"],
  ["I love watching sunsets by the beach.", "chat"],
  ["My favorite season is autumn because of the beautiful colors.", "chat"],
  ["I recently started doing yoga to improve my flexibility.", "chat"],
  ["Last weekend, I attended a friend's wedding.", "chat"],
  ["I've been reading a fascinating book about history lately.", "chat"],
  ["I enjoy listening to classical music while working.", "chat"],
  ["My favorite hobby is photography.", "chat"],
  ["I've always wanted to learn how to dance salsa.", "chat"],
  ["My family and I went hiking in the mountains last summer.", "chat"],
  ["I love visiting art galleries and museums.", "chat"],
  ["I recently adopted a rescue dog, and he's so adorable.", "chat"],
  ["My favorite type of cuisine is Italian.", "chat"],
  ["I love going to concerts and experiencing live music.", "chat"],
  ["I find painting to be a relaxing activity.", "chat"],
  ["One of my favorite movies is The Shawshank Redemption.", "chat"],
  ["I enjoy playing board games with my friends.", "chat"],
  ["I've always admired Vincent van Gogh's art.", "chat"],
  ["I enjoy gardening and growing my own vegetables.", "chat"],
  ["I find stargazing to be a fascinating hobby.", "chat"],
  ["My favorite book is To Kill a Mockingbird.", "chat"],
  ["I love the smell of freshly brewed coffee in the morning.", "chat"],
  ["I find baking to be a fun and creative activity.", "chat"],
  ["I'm learning a new language to challenge myself.", "chat"],
  ["I've been practicing meditation to help with stress.", "chat"],
  ["I enjoy volunteering at the local animal shelter.", "chat"],
  ["My favorite flower is the rose because of its beauty and fragrance.", "chat"],
  ["I love exploring new places and trying new food.", "chat"],
  ["I find swimming to be a great form of exercise.", "chat"],
  ["One of my favorite childhood memories is going on family vacations.", "chat"],
  ["I've always been fascinated by astronomy.", "chat"],
  ["I enjoy watching documentaries about nature and wildlife.", "chat"],
  ["My favorite type of tea is green tea.", "chat"],
  ["I've recently taken up pottery as a new hobby.", "chat"],
  ["I enjoy going for long bike rides on weekends.", "chat"],
  ["I love listening to jazz music while cooking dinner.", "chat"],
  ["I'm a big fan of mystery novels.", "chat"],
  ["I find spending time in nature to be very therapeutic.", "chat"],
  ["I enjoy going to comedy shows and having a good laugh.", "chat"],
  ["I love the feeling of accomplishment after finishing a challenging puzzle.", "chat"],
  ["I find the sound of rain to be very calming.", "chat"],
  ["I enjoy visiting local farmers' markets on weekends.", "chat"],
  ["I love the smell of fresh flowers in the spring.", "chat"],
  ["My favorite holiday is Christmas because of the festive atmosphere.", "chat"]
]

In [3]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Prepare the dataset
texts = [item[0] for item in data]
labels = [item[1] for item in data]
label_map = {'vision': 0, 'chat': 1}
labels = [label_map[label] for label in labels]

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Tokenize the text
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Create the custom dataset
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

# Create the Trainer
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    learning_rate=5e-5,
    save_total_limit=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Save a checkpoint at the end of each epoch
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()


ModuleNotFoundError: No module named 'sklearn'

In [42]:
from transformers import pipeline

# Load the fine-tuned model
model_path = './results/checkpoint-150'
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
classifier = pipeline('text-classification', model=model_path, tokenizer=tokenizer)

def command_filter(prompt):
    # Classify the input prompt
    result = classifier(prompt)
    command_id = int(result[0]['label'].split('_')[-1])
    command = {0: 'vision', 1: 'chat'}[command_id]

    return command
    
# Example prompts

prompt1 = "How many people live in London?"
prompt2 = "I need you to tell me about powerlifting"
prompt3 = "Can you see me?"
prompt4 = "What do you see in this image?"

print(f'{prompt1} : {command_filter(prompt1)}')
print(f'{prompt2} : {command_filter(prompt2)}')
print(f'{prompt3} : {command_filter(prompt3)}')
print(f'{prompt4} : {command_filter(prompt4)}')

How many people live in London? : chat
I need you to tell me about powerlifting : chat
Can you see me? : vision
What do you see in this image? : vision
